In [ ]:
import xgboost
from xgboost import XGBClassifier
import numpy as np
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split
import pandas as pd
from sklearn.metrics import *
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import confusion_matrix

In [ ]:
dataframe = pd.read_csv('Data/Churn_Modelling.csv')

In [ ]:
dataframe.head()

In [ ]:
plt.figure(figsize=(9,7))
sns.heatmap(dataframe.corr())

In [ ]:

dataframe,test = train_test_split(dataframe,test_size=0.05,random_state=0)

In [ ]:
test.to_csv('Data/test.csv')

In [ ]:
X = dataframe.drop(['Exited'],axis=1)
y = dataframe['Exited']

In [ ]:
X.head()

In [ ]:
# function to remove cols having less correlation
num_col = (X.columns[X.dtypes=='int64'])
for i in num_col:
    if (dataframe.corr()['Exited'][i] < -0.02):
        X=X.drop([i],axis=1)

In [ ]:
X.head()

In [ ]:
X.shape

In [ ]:
X.nunique()

In [ ]:
X.drop(['Surname','CustomerId'],axis=1,inplace=True)

In [ ]:
X.columns

In [ ]:
class pre_process:
    
    def __init__(self):
        self.cols = ['Geography','Gender']
        self.needed_cols= ['Age', 'Tenure', 'Geography','Gender','Balance', 'HasCrCard', 'EstimatedSalary']
        self.added_cols=[]
    
    def pre_processing(self,X):
        models= []
        added_cols = []
        X = X[self.needed_cols]
#         print(X.head())
        for col in self.cols:
            sorted_col = np.sort(X[col].unique())
            lb = LabelBinarizer()
            lb.fit(X[col])
            binarized = lb.transform(X[col])
            models.append(lb)
            for i in range(0,len(sorted_col)-1):
                X[sorted_col[i]]=binarized[:,i]
                added_cols.append(sorted_col[i]+'_lbinarizer')
            X.drop([col],axis=1,inplace=True)
#         print(added_cols)
        # scalers
        for cols in X.columns:
            if (cols+'_lbinarizer') not in added_cols:
                ss = StandardScaler()
                ss.fit(np.array(X[cols]).reshape(-1,1))
                X[cols] = ss.transform(np.array(X[cols]).reshape(-1,1))
                models.append(ss)
                added_cols.append(cols+'_SScaler')
#         self.added_cols = added_cols
        return X, models, added_cols


In [ ]:
prep = pre_process()
X, models,added_cols=prep.pre_processing(X)

In [ ]:
X.head()

In [ ]:
X.columns

In [ ]:
X.columns

In [ ]:
y.head()

In [ ]:
X_train,X_cross,y_train,y_cross = train_test_split(X,y,test_size=0.1,random_state=0)

In [ ]:
xgc = XGBClassifier()

In [ ]:
xgc.fit(X_train,y_train)

In [ ]:
accuracies = cross_val_score(estimator=xgc,X=X_train,y= y_train,cv=7)

In [ ]:
params = {'n_estimators':[25,50,100,150,200],'learning_rate':[0.1,0.05, 0.02, 0.8,0.9] }
gsv = GridSearchCV(estimator=xgc,param_grid=params,cv=10)

In [ ]:
gsv.fit(X_train,y_train)

In [ ]:
gsv.best_params_

In [ ]:
gsv.best_score_

In [ ]:
accuracies

In [ ]:
y_pred = xgc.predict(X_cross)

In [ ]:
len(y_pred)

In [ ]:
confusion_matrix(y_cross,y_pred)

In [ ]:
f1_score(y_cross,y_pred)

In [ ]:
# models.append(pre)
models.append(xgc)

In [ ]:
with open('pickle/model.pkl','wb') as f:
    pickle.dump(str(added_cols),f)
    pickle.dump(models,f)